In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd
from urllib.parse import urljoin


In [ ]:
def parse_full_review(url):
    """
    Парсит полную страницу отзыва
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Основной контейнер отзыва
        review_block = soup.select_one('.reviewBlock')
        if not review_block:
            print(f"Не найден контейнер отзыва для {url}")
            return None
        
        # Извлекаем данные
        review_data = {}
        
        # Автор
        author_element = review_block.select_one('.reviewer a')
        review_data['author'] = author_element.get_text(strip=True) if author_element else 'N/A'
        
        # Рейтинг (из мета-тега)
        rating_meta = review_block.select_one('meta[itemprop="ratingValue"]')
        review_data['rating'] = rating_meta['content'] if rating_meta else 'N/A'
        
        # Дата публикации
        date_element = review_block.select_one('.dtreviewed')
        review_data['full_date'] = date_element.get_text(strip=True) if date_element else 'N/A'
        
        # Заголовок отзыва
        title_element = review_block.select_one('.reviewTitle a')
        review_data['full_title'] = title_element.get_text(strip=True) if title_element else 'N/A'
        
        # Полный текст отзыва
        review_body = review_block.select_one('[itemprop="reviewBody"]')
        if review_body:
            # Удаляем скрипты и стили
            for script in review_body(["script", "style"]):
                script.decompose()
            
            # Получаем чистый текст
            full_text = review_body.get_text(separator='\n', strip=True)
            # Убираем лишние пробелы и переносы
            full_text = re.sub(r'\n\s*\n', '\n\n', full_text)
            review_data['full_text'] = full_text
            
            # Дополнительно: извлекаем отдельные элементы текста
            paragraphs = review_body.find_all('p')
            review_data['paragraphs_count'] = len(paragraphs)
            
            # Извлекаем цитаты
            quotes = review_body.select('blockquote p')
            review_data['quotes'] = [quote.get_text(strip=True) for quote in quotes]
            
            # Извлекаем списки (плюсы/минусы)
            lists = review_body.find_all(['ul', 'ol'])
            review_data['lists_count'] = len(lists)
        else:
            review_data['full_text'] = 'N/A'
            review_data['paragraphs_count'] = 0
            review_data['quotes'] = []
            review_data['lists_count'] = 0
        
        # Количество изображений в отзыве
        images = review_block.select('.inline-image img')
        review_data['images_count'] = len(images)
        review_data['image_urls'] = []
        
        for img in images:
            img_src = img.get('src') or img.get('data-original')
            if img_src and not img_src.startswith('data:'):
                if img_src.startswith('//'):
                    img_src = 'https:' + img_src
                elif img_src.startswith('/'):
                    img_src = 'https://irecommend.ru' + img_src
                review_data['image_urls'].append(img_src)
        
        # Вердикт (рекомендует/не рекомендует)
        verdict_element = review_block.select_one('.verdict')
        review_data['verdict'] = verdict_element.get_text(strip=True) if verdict_element else 'N/A'
        
        # Дополнительная информация
        review_data['url'] = url
        review_data['timestamp'] = time.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"Обработан отзыв: {review_data['full_title'][:50]}...")
        return review_data
        
    except Exception as e:
        print(f"Ошибка при парсинге {url}: {e}")
        return None

def process_all_reviews(df, start_from=0, delay=2):
    """
    Обрабатывает все ссылки на отзывы из DataFrame
    """
    full_reviews_data = []
    
    for idx, row in df.iterrows():
        if idx < start_from:
            continue
            
        url = row['full_review_link']
        print(f"Обработка отзыва {idx + 1}/{len(df)}: {url}")
        
        review_data = parse_full_review(url)
        if review_data:
            # Объединяем с исходными данными
            combined_data = {**row.to_dict(), **review_data}
            full_reviews_data.append(combined_data)
        
        # Задержка между запросами
        time.sleep(delay)
    
    return full_reviews_data

# Основной код
if __name__ == "__main__":
    # Загружаем CSV с собранными отзывами
    df = pd.read_csv('chips_reviews.csv')
    
    print(f"Всего отзывов для обработки: {len(df)}")
    
    # Обрабатываем все отзывы
    all_full_reviews = process_all_reviews(df, start_from=0, delay=2)
    
    # Сохраняем результаты
    if all_full_reviews:
        full_df = pd.DataFrame(all_full_reviews)
        
        # Сохраняем в CSV
        full_df.to_csv('chips_reviews_full.csv', index=False, encoding='utf-8')
        
        # Сохраняем в JSON
        full_df.to_json('chips_reviews_full.json', orient='records', force_ascii=False, indent=2)
        
        print(f"\nУспешно обработано {len(all_full_reviews)} полных отзывов")
        print("Данные сохранены в chips_reviews_full.csv и chips_reviews_full.json")
        
        # Выводим пример данных
        print("\nПример данных:")
        print(full_df[['author', 'rating', 'full_title', 'images_count']].head())
    else:
        print("Не удалось обработать ни одного отзыва")

In [ ]:
df = pd.read_csv('chips_reviews_full.csv')
df.head(2)

In [ ]:
print(df['full_text'][0])